# Uber:   Given locations of customers and drivers, make an "optimal" assignment.

In [1]:
from pprint import pprint

In [2]:
# Let's consider two customers, C1 and C2, we provide their locations:  [lat,lng]
C1_location = [33.780775, -84.386301]
C2_location = [33.778988, -84.363791]

# And two available drivers, D1 and D2
D1_location = [33.793711, -84.317408]
D2_location = [33.776812, -84.356153]

### We will use the `geopy` package to compute distances between two locations (it considers the curvature of the earth for us!) The documentation can be found here:   https://pypi.org/project/geopy/
#### If you are running locally you will need to install geopy with the command at the terminal:   `pip install geopy`

In [3]:
from geopy.distance import geodesic

In [4]:
# We will store distances in a dictionary, with key 'CustomerName_DriverName'
distances = {}
distances['C1_D1'] = geodesic(C1_location,D1_location).miles
distances['C1_D2'] = geodesic(C1_location,D2_location).miles

distances['C2_D1'] = geodesic(C2_location,D1_location).miles
distances['C2_D2'] = geodesic(C2_location,D2_location).miles

pprint(distances)

{'C1_D1': 4.063663522199328,
 'C1_D2': 1.7564928121576306,
 'C2_D1': 2.8556454250592753,
 'C2_D2': 0.46447745846674}


### We are now ready to form a Linear Program to find an optimal assignment.  

### There are a number of packages to do optimization, we will use `PuLP` that is open source and free, but works much the same way as a commercial product.   The documentation is here:   https://pypi.org/project/PuLP/
#### If you are running locally you will need to install at the terminal with: `pip install pulp`

In [5]:
from pulp import *

### We use the PuLP function `LpProblem` to define a model instance, with an objective we will want to Minimize

In [6]:
model = LpProblem("Uber Assignment",LpMinimize)
# Let's look at our model, nothing much here yet.
pprint(model)

Uber Assignment:
MINIMIZE
None
VARIABLES



### We will define a varible for each Customer/Driver pair.  The variable names will be of the form, `CustName_DriverName`,  so `C1_D2` for example.  

In [7]:
# Each arc variable is Binary 0,1
C1_D1 = pulp.LpVariable('C1_D1', cat='Binary')
C1_D2 = pulp.LpVariable('C1_D2', cat='Binary')
C2_D1 = pulp.LpVariable('C2_D1', cat='Binary')
C2_D2 = pulp.LpVariable('C2_D2', cat='Binary')


In [8]:
# Each of these variables is a special class defined by the PuLP system.   
# We just nod our head in approval, not caring that we don't understand the inner workings of this class.
type(C1_D1)

pulp.pulp.LpVariable

### We can now add our objective function to `model` which is to Minimize the sum of distance travelled

In [9]:
model +=  distances['C1_D1']*C1_D1 + distances['C1_D2']*C1_D2 +   distances['C2_D1']*C2_D1 + distances['C2_D2']*C2_D2
pprint(model)

Uber Assignment:
MINIMIZE
4.063663522199328*C1_D1 + 1.7564928121576306*C1_D2 + 2.8556454250592753*C2_D1 + 0.46447745846674*C2_D2 + 0.0
VARIABLES
0 <= C1_D1 <= 1 Integer
0 <= C1_D2 <= 1 Integer
0 <= C2_D1 <= 1 Integer
0 <= C2_D2 <= 1 Integer



### We now need our "node" constraints, each customer is assignned to exactly one driver, and each driver is assigned exactly one customer.  

### Now we add these constraints to our `model`

In [10]:
# Add constraint for each customer
#  First we keep each constraint in a variable (this will help us get some info about it later)
#  Then we add it to our model with a name in quotes 
C1_const = C1_D1 + C1_D2 == 1
model += C1_const, "C1" 
C2_const = C2_D1 + C2_D2 == 1
model += C2_const, "C2" 
# Add constraint for each driver 
D1_const = C1_D1 + C2_D1 == 1
model += D1_const, "D1" 
D2_const = C1_D2 + C2_D2 == 1
model += D2_const, "D2" 

In [11]:
# Let's inspect our model to make sure it looks right
model

Uber Assignment:
MINIMIZE
4.063663522199328*C1_D1 + 1.7564928121576306*C1_D2 + 2.8556454250592753*C2_D1 + 0.46447745846674*C2_D2 + 0.0
SUBJECT TO
C1: C1_D1 + C1_D2 = 1

C2: C2_D1 + C2_D2 = 1

D1: C1_D1 + C2_D1 = 1

D2: C1_D2 + C2_D2 = 1

VARIABLES
0 <= C1_D1 <= 1 Integer
0 <= C1_D2 <= 1 Integer
0 <= C2_D1 <= 1 Integer
0 <= C2_D2 <= 1 Integer

In [12]:
# Let's solve the model and make sure it's status is good
model.solve()
print("Status:", LpStatus[model.status])

Status: Optimal


In [13]:
# Here is the solution
for v in model.variables():
    print(v.name, "=", v.varValue)

C1_D1 = 1.0
C1_D2 = 0.0
C2_D1 = 0.0
C2_D2 = 1.0


## Shadow Prices
### For those of you familiar with Linear Programming, you may wonder how to get the shadow prices on our constraints,  here they are. 
### How might they be of use here? 

In [14]:
print(C1_const, "  shadow price:", C1_const.pi)
print(C2_const, "  shadow price:", C2_const.pi)
print(D1_const, "  shadow price:", D1_const.pi)
print(D2_const, "  shadow price:", D2_const.pi)

C1_D1 + C1_D2 = 1   shadow price: 0.0
C2_D1 + C2_D2 = 1   shadow price: 0.0
C1_D1 + C2_D1 = 1   shadow price: 0.0
C1_D2 + C2_D2 = 1   shadow price: 0.0


## Homework/Classroom work

- Can you reformulate the problem so that we minimize the worst case?  This would, it seems in this case, change the assignment we got originally.
- What if there are not an equal number of drivers and customers, how do we reformulate our model?
- What if there are some customers who have waited considerably longer than others, or drivers who have waited longer ... how might your model formulation account for this?